In [1]:
import os
import torch

# set params
ae_path = '/home/jeeves/JJ_Projects/klara_models/flux_dev/train-model/ae.safetensors'
transformer_path = '/home/jeeves/JJ_Projects/klara_models/flux_dev/train-model/flux1-dev.safetensors'
os.environ['FLUX_DEV'] = transformer_path
os.environ['AE'] = ae_path

width, height = 1024, 1024
device = torch.device('cuda')
dtype = torch.bfloat16
seed = 42
prompt = "a cute boy running on the street"

guidance = 3.5
steps = 20

In [ ]:
from flux.util import load_ae, load_clip, load_flow_model, load_t5

# load models
ae = load_ae(name='flux-dev', device=device, hf_download=False)
clip = load_clip(device=device)
t5 = load_t5(device=device, max_length=512)
transformer = load_flow_model(name='flux-dev', device=device, hf_download=False)
torch.cuda.empty_cache()



In [ ]:
import torch
from PIL import Image
from flux.modules.autoencoder import AutoEncoder
from flux.sampling import prepare, get_schedule, denoise, unpack
from torchvision import transforms
import einops
import numpy as np

@torch.inference_mode()
def encode(img: Image.Image, ae: AutoEncoder, device=torch.device('cuda')):
    img_np = np.array(img)
    img_tensor = (torch.from_numpy(img_np).permute(2, 0, 1).float() / 127.5 - 1).unsqueeze(0)
    img_tensor = img_tensor.to(device=device)
    latent = ae.encode(img_tensor)
    return latent.to(dtype=torch.bfloat16)

@torch.inference_mode()
def decode(latent: torch.Tensor, ae: AutoEncoder):
    img_tensor = ae.decode(latent)
    img_tensor = einops.rearrange(img_tensor[0], "c h w -> h w c")
    img_tensor = img_tensor.clamp(-1, 1)
    img = Image.fromarray((127.5 * (img_tensor + 1.0)).cpu().byte().numpy())
    return img


steps = 4
with torch.inference_mode():
    input_image = Image.open('test.jpg')

    latent = encode(input_image, ae)
    print(latent.shape)

    inputs_dict = prepare(t5, clip, latent, prompt="catoon style, a man hugging with a woman")

    img = inputs_dict['img']
    img_ids = inputs_dict['img_ids']
    txt = inputs_dict['txt']
    txt_ids = inputs_dict['txt_ids']
    vec = inputs_dict['vec']

    timesteps = get_schedule(num_steps=steps, image_seq_len=latent.shape[1])
    inverse_timesteps = timesteps[:: -1]
    print(timesteps)
    print(inverse_timesteps)

    # gamma = 0.0
    # target_noise = torch.randn(img.shape, device=img.device, dtype=torch.float32)
    # target_noise = torch.randn(img.shape, device=img.device, dtype=torch.bfloat16)

    guidance = 7

    use_rf_solver = False
    guidance_vec = torch.full((img.shape[0], ), guidance, device=device, dtype=img.dtype)
    for t_curr, t_prev in zip(inverse_timesteps[: -1], inverse_timesteps[1: ]):
        t_vec = torch.full((img.shape[0], ), t_curr, device=device, dtype=img.dtype)
        pred = transformer(
            img = img, img_ids = img_ids,
            txt = txt, txt_ids = txt_ids,
            y = vec,
            timesteps=t_vec,
            guidance=guidance_vec
        )

        img = img + (t_prev - t_curr) * pred

    res = denoise(transformer, img=img, img_ids=img_ids, txt=txt, txt_ids=txt_ids, vec=vec, timesteps=timesteps, guidance=guidance)
    with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
        rect_img = decode(unpack(res, height, width), ae)
        latent_img = decode(unpack(img, height, width), ae)
        vae_rect_img = decode(latent, ae)

    vae_rect_img.save('vae_rect.png')
    rect_img.save('rect.png')
    latent_img.save('latent.png')



In [ ]:

from PIL import Image
import math
import einops
from typing import Callable
from torchvision import transforms
# 现在我们需要一步步自己实现下面的采样函数
# from flux.sampling import get_noise, prepare, denoise, get_schedule, unpack

def get_indices(mask_path: str):
    mask_image = Image.open(mask_path)
    vae_scale_factor = 8
    patch_size = 2

    width, height = mask_image.size

    man_mask_resized = mask_image.resize((width // (vae_scale_factor * patch_size), height // (vae_scale_factor * patch_size)))

    print(man_mask_resized.size)
    man_mask_resized_tensor = transforms.ToTensor()(man_mask_resized)

    print(man_mask_resized_tensor.shape)

    index = torch.nn.Flatten()(man_mask_resized_tensor).squeeze()

    # 将一维索引扩展到与A和B相同的形状
    index = index.view(1, -1, 1).to(dtype=torch.int64)  # 现在index的形状是[1, 4096, 1]
    return index

def time_shift(mu: float, sigma: float, t: torch.Tensor):
    return math.exp(mu) / (math.exp(mu) + (1 / t - 1) ** sigma)

def get_lin_function(
    x1: float = 256, y1: float = 0.5, x2: float = 4096, y2: float = 1.15
) -> Callable[[float], float]:
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return lambda x: m * x + b

steps = 4

with torch.inference_mode():
    # 获取噪声
    # x_T = get_noise(num_samples=1, height=height, width=width, device=device, dtype=dtype, seed=seed)
    x_T = torch.randn(
        1,
        16,  # channels
        # 需要能够被 16 整除 ？ TODO  这里的 2 是为了下面能被 块的尺寸 pw/ph 整除？
        2 * math.ceil(height / 16),  
        2 * math.ceil(width / 16),
        device=device,
        dtype=dtype,
        generator=torch.Generator(device=device).manual_seed(seed),
    )
    print("x_T shape:", x_T.shape)  # [1, 16, 128, 128]

    # 准备输入：img，img_ids，txt，txt_ids
    # inputs_dict = prepare(t5, clip, x_T, prompt)
    bs, c, h, w = x_T.shape

    # 图块化  [1, 16, 128(64 * 2), 128] -> [1, 4096(64 * 64), 64(16 * 2 * 2)]   (bs, img_token_seq_len, dim_in)
    # ph/pw 块高、宽, 都取 2。
    # 这里 patch 的宽高都是写死 2 的话，是不是图块化/反图块化写在模型内部比较合理？
    x_T_patchified = einops.rearrange(x_T, "b c (h ph) (w pw) -> b (h w) (c ph pw)", ph=2, pw=2)
    img = x_T_patchified

    # img_ids TODO   这是干嘛？？？？？？？？？
    img_ids = torch.zeros(h // 2, w // 2, 3)
    img_ids[..., 1] = img_ids[..., 1] + torch.arange(h // 2)[:, None]
    img_ids[..., 2] = img_ids[..., 2] + torch.arange(w // 2)[None, :]
    img_ids = einops.repeat(img_ids, "h w c -> b (h w) c", b=bs)
    img_ids = img_ids.to(device)
    print('img_ids shape:', img_ids.shape)  # [bs, img_token_seq_len, 3] [1, 4096, 3]

    # 文本条件嵌入
    txt = t5(prompt)
    print('txt shape:', txt.shape)  # [bs, t5_seq_len, t5_dim]  [1, 512, 4096]

    # txt_ids
    txt_ids = torch.zeros(bs, txt.shape[1], 3)
    txt_ids = txt_ids.to(device)
    print('txt_ids shape:', txt_ids.shape)  # [bs, t5_seq_len, 3]   [1, 512, 3]

    vec = clip(prompt)
    print('clip vec shape:', vec.shape)  # [bs, clip_dim]    [1, 768]

    # 获取采样时间步
    # timesteps = get_schedule(steps, inputs_dict["img"].shape[1], shift=True)
    timesteps = torch.linspace(1, 0, steps+1)
    shift = True
    if shift:
        base_shift = 0.5
        max_shift = 1.15
        image_seq_len = img.shape[1]
        mu = get_lin_function(y1=base_shift, y2=max_shift)(image_seq_len)
        timesteps = time_shift(mu, 1.0, timesteps)

    timesteps = timesteps.tolist()

    print('timesteps')
    print(timesteps)

    # 去噪循环
    # x = denoise(transformer, **inputs_dict, guidance=guidance_scale, timesteps=timesteps)

    # cfg 蒸馏的结果，直接将 guidance 标量值输入模型。只有 dev 模型有 guidance，schnell 模型没有,
    guidance_vec = torch.full((img.shape[0], ), guidance, device=device, dtype=img.dtype)
    for t_curr, t_prev in zip(timesteps[: -1], timesteps[1: ]):  # TODO 为啥这样写，和 RF 采样公式有关？？？
        t_vec = torch.full((img.shape[0], ), t_curr, device=device, dtype=img.dtype)
        pred = transformer(
            img=img, img_ids=img_ids,
            txt=txt, txt_ids=txt_ids,
            y=vec,
            timesteps=t_vec,
            guidance=guidance_vec
        )

        img = img + (t_prev - t_curr) * pred   # RF 的采样公式 ？？？？？？？？
    x = img

    print("x_0 shape:", x.shape)
    rand_x = torch.randn(*x.shape).to(device=device)
    print('rand x shape:', rand_x.shape)

    index = get_indices('mask_man.png').to(device=device)
    x = x * (index == 0) + rand_x * (index == 1)

    # 反图块化
    # x = unpack(x.float(), height, width)
    x = einops.rearrange(x, "b (h w) (c ph pw) -> b c (h ph) (w pw)", h=64, w=64, ph=2, pw=2)
    print("x shape after unpacked", x.shape)

    with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
        x = ae.decode(x)

    x = x.clamp(-1, 1)
    x = einops.rearrange(x[0], "c h w -> h w c")
    img = Image.fromarray((127.5 * (x + 1.0)).cpu().byte().numpy())

img
